In [1]:
#notebook for scraping
#dependencies 

import os 
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/summyabakshi/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [3]:
#website to scrape 
url = 'https://redplanetscience.com/'
#retrieve page wiht splinter module - this page will need to stay open to retreive the parsed text
browser.visit(url)


In [4]:
html = browser.html
#parse with soup
soup = BeautifulSoup(html, 'html.parser')

#### Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [5]:
#parent divs for latest article title and parargaraph

title = soup.find('div', class_='content_title').text
print(title)
para = soup.find('div', class_='article_teaser_body').text
print(para)



NASA's Curiosity Mars Rover Snaps Its Highest-Resolution Panorama Yet
To go along with the stunning 1.8-billion-pixel image, a new video offers a sweeping view of the Red Planet.


In [6]:
#remeber to quit broswer 
#browser.quit()

#### JPL Mars Space Images - Featured Image

In [7]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [8]:
jpl_url = 'https://spaceimages-mars.com/'
#open the website for scraping
browser.visit(jpl_url)



In [9]:
html = browser.html
#parse
soup = BeautifulSoup(html, 'html.parser')


In [10]:
featured_img_url = soup.find('img', class_="headerimage fade-in")['src']
featured_img_url = (jpl_url + featured_img_url)
print(featured_img_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [11]:
#browser.quit()

### Pandas Scraping 

#### Mars Facts

In [12]:
mars_facts_url = 'https://galaxyfacts-mars.com/'
#create a tablular structure out of the URL
table = pd.read_html(mars_facts_url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [13]:
facts_df = table[0]
facts_df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [14]:
facts_df = facts_df.rename(columns={0:'Mars-Earth Comparison', 1: 'Mars', 2:'Earth' })
facts_df

,Mars-Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
facts_df = facts_df.drop([0])


In [16]:
facts_df = facts_df.set_index(['Mars-Earth Comparison'])
facts_df.head()

,Mars,Earth
Mars-Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


In [17]:
#convert table to html string
html_table = facts_df.to_html('mars_facts_table.html')
html_table
#do they want us to export the HTML as well ?

#### Mars Hemispheres

In [18]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [19]:
#hesmisphere website for scraping
img_url = 'https://marshemispheres.com/'
#retreive page with requests module 
browser.visit(img_url)
#create soup object with parser
html = browser.html
#parse
soup = BeautifulSoup(html, 'html.parser')


In [21]:
hem_links = soup.find_all('div', class_ = 'description')
print(hem_links)

hem_list = []

for i in range(len(hem_links)):
    hem_dict = {}
    #browser.find_by_css('a.product-item img')[i].click()
               
    img_html = hem_links[i].find(class_='itemLink').get('href')
    print(img_html)
    element = browser.hem_links.find_by_text('Sample').first()
    print(element)
    hem_dict['img_url']= element['href']
    hem_dict['title']= browser.find_by_css('h2.title').text
    print(hem_dict)
    hem_list.append(hem_dict)
    browser.back()
    

               

[]


In [ ]:
hem_list

In [ ]:
results = soup.find('div', class_ ='collapsible results')
#each_hem = results.find_all('div',class_='item')
each_hem

In [ ]:
hemisphere_dict = []

for result in results:
    title = result.find('h3').text
    h_url = result.find('img', class_ = 'thumb')['src']
    full_url = (img_url+h_url)
    print(title)
    print(full_url)
    print('---------')
    hemisphere_dict.append({'Image Title': title ,'Image URL': full_url})
#only printing the small images 
hemisphere_dict
    

In [ ]:
mars_dict = {
    'titles': title,
    'para': para,
    'featured img': featured_img_url, 
    'hemisphere imgs': hemisphere_dict 
}
mars_dict